In [0]:
dbutils.widgets.removeAll()

In [0]:
#input  path(src, dest , arch) from widget
dbutils.widgets.text("source_path","/FileStore/tables/source")
dbutils.widgets.text("destination_path" , "FileStore/tables/destination")
dbutils.widgets.text("archieve_path" , "FileStore/tables/archieve")
dbutils.widgets.text("filename" , "Superstore1.csv")
dbutils.widgets.text("filetype" , "csv")



In [0]:
#main method( read input widgets and functions)
class task:
    def __init__(self, params):
        self.source_path = params["source_path"]
        self.dest_path = params["dest_path"]
        self.filename = params["filename"]
        self.filetype = params["filetype"]
        
    def read_csv(self,source_path ,filetype , filename):
            return spark.read.options(header='True' , inferSchema='True' ).format(filetype).load(source_path +"/" + filename)
        
    def null_value(self ):
        df=self.read_csv(self.source_path , self.filetype , self.filename)
        df=df.filter(F.col("Order ID").isNotNull() & F.col("Order Date").isNotNull() & F.col("Ship Mode").isNotNull() & F.col("Customer ID").isNotNull() )
        df.show()
        return df
    
    
    def date_formating(self):
        df= self.null_value()
        df= df.withColumn("Order Date", F.to_date(F.col("Order Date"), "dd-MM-yyyy"))
        df.show()
        return df
    
    def myfunc(self):
        print("reading file from path :{}" .format(self.source_path))
        df=self.date_formating()
        print("df count :{}".format(df.count()))
        df= df.withColumn("year" , F.year(F.current_date()))
        df= df.withColumn("month" , F.month(F.current_date()))
        df= df.withColumn("day" , F.dayofmonth(F.current_date()))
        df=df.withColumn("hour" , F.hour(F.current_timestamp()))
        df.printSchema()
        self.write_csv(df , self.dest_path ,)
        return df
        
    def internal_table(self):
        df= self.myfunc()
        df_final = (df.withColumn("row_id", df["Row ID"]).drop("Row ID")
                    .withColumn("order_id", df["Order ID"]).drop("Order ID")
                    .withColumn("order_date", df['Order Date']).drop("Order Date")
                    .withColumn("customer_name", df["Customer Name"]).drop("Customer Name")
                    .withColumn("year" , F.year(F.current_date()))
                    .withColumn("month" , F.month(F.current_date()))
                    .withColumn("day" , F.dayofmonth(F.current_date()))
                    .withColumn("hour" , F.hour(F.current_timestamp()))
                    .drop("Ship Date", "Ship Mode" ,"Customer ID" , "Segment" ,"Country","City" ,"State")
              .distinct()
            ) 
        df_final.show()
        df_final.createOrReplaceTempView("df_final_view")
        spark.sql("CREATE TABLE IF NOT EXISTS salesTable_internal  AS SELECT * FROM df_final_view")
        
    def external_table(self):
        df= self.myfunc()
        df_final = (df.withColumn("row_id", df["Row ID"]).drop("Row ID")
                    .withColumn("order_id", df["Order ID"]).drop("Order ID")
                    .withColumn("order_date", df['Order Date']).drop("Order Date")
                    .withColumn("customer_name", df["Customer Name"]).drop("Customer Name")
                    .withColumn("year" , F.year(F.current_date()))
                    .withColumn("month" , F.month(F.current_date()))
                    .withColumn("day" , F.dayofmonth(F.current_date()))
                    .withColumn("hour" , F.hour(F.current_timestamp()))
                    .drop("Ship Date", "Ship Mode" ,"Customer ID" , "Segment" ,"Country","City" ,"State")
              .distinct()
            ) 
        df_final.show()
        df_final.createOrReplaceTempView("df_final_external")
        spark.sql("CREATE EXTERNAL TABLE IF NOT EXISTS salesTable_external PARTITIONED BY (year , month , day, hour ) LOCATION '/FileStore/tables/salesTable_external'AS SELECT* FROM df_final_external")
        
    
    def write_csv(self , df, dest_path):
        df.write.option("header",True).partitionBy("year" , "month" ,"day" , "hour").mode("Append") .parquet(dest_path)
        print("data written succesfully:{}".format(dest_path))




In [0]:
#call main function
source= dbutils.widgets.get("source_path")
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession.builder.appName("Spark Dataframes").getOrCreate()
d ={"source_path":dbutils.widgets.get("source_path") ,
    "dest_path":dbutils.widgets.get("destination_path"),
    "filename": dbutils.widgets.get("filename"),
    "filetype":dbutils.widgets.get("filetype") }

t=task(d)
t1=t.myfunc()
t1.show()
t.external_table()




reading file from path :/FileStore/tables/source


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-4468627208457168> in <module>
     10 
     11 t=task(d)
---> 12 t1=t.myfunc()
     13 t1.show()
     14 t.external_table()

<command-4468627208457167> in myfunc(self)
     25     def myfunc(self):
     26         print("reading file from path :{}" .format(self.source_path))
---> 27         df=self.date_formating()
     28         print("df count :{}".format(df.count()))
     29         df= df.withColumn("year" , F.year(F.current_date()))

<command-4468627208457167> in date_formating(self)
     18 
     19     def date_formating(self):
---> 20         df= self.null_value()
     21         df= df.withColumn("Order Date", F.to_date(F.col("Order Date"), "dd-MM-yyyy"))
     22         df.show()

<command-4468627208457167> in null_value(self)
     11 
     12     def null_value(self ):
---> 13         df=self.read_csv(self.source_pa

In [0]:
dbutils.fs.ls("FileStore/tables/destination/output1.parquet/year=2022/month=7/day=25/hour=16")

Out[64]: [FileInfo(path='dbfs:/FileStore/tables/destination/output1.parquet/year=2022/month=7/day=25/hour=16/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1658767103000),
 FileInfo(path='dbfs:/FileStore/tables/destination/output1.parquet/year=2022/month=7/day=25/hour=16/_committed_6852104357179331242', name='_committed_6852104357179331242', size=124, modificationTime=1658767102000),
 FileInfo(path='dbfs:/FileStore/tables/destination/output1.parquet/year=2022/month=7/day=25/hour=16/_started_6852104357179331242', name='_started_6852104357179331242', size=0, modificationTime=1658767101000),
 FileInfo(path='dbfs:/FileStore/tables/destination/output1.parquet/year=2022/month=7/day=25/hour=16/part-00000-tid-6852104357179331242-dac42428-78b8-4c08-9f12-f18d6ff00f09-121-1.c000.snappy.parquet', name='part-00000-tid-6852104357179331242-dac42428-78b8-4c08-9f12-f18d6ff00f09-121-1.c000.snappy.parquet', size=4054, modificationTime=1658767102000)]

In [0]:
%sql
select * from salesTable_internal
-- drop table salesTable_internal

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: Table or view not found: salesTable_internal; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [salesTable_internal], [], false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$2(CheckAnalysis.scala:138)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$2$adapted(CheckAnalysis.scala:105)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:358)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:357)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:357)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:357)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$1(CheckAnalysis.scala:105)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:80)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis(CheckAnalysis.scala:100)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis$(CheckAnalysis.scala:100)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.checkAnalysis(Analyzer.scala:222)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.$anonfun$executeAndCheck$1(Analyzer.scala:276)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.markInAnalyzer(AnalysisHelper.scala:331)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.executeAndCheck(Analyzer.scala:273)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$analyzed$1(QueryExecution.scala:128)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:80)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:151)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhase$1(QueryExecution.scala:265)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:968)
	at org.apache.spark.sql.execution.QueryExecution.executePhase(QueryExecution.scala:265)
	at org.apache.spark.sql.execution.QueryExecution.analyzed$lzycompute(QueryExecution.scala:129)
	at org.apache.spark.sql.execution.QueryExecution.analyzed(QueryExecution.scala:126)
	at org.apache.spark.sql.execution.QueryExecution.assertAnalyzed(QueryExecution.scala:118)
	at org.apache.spark.sql.Dataset$.$anonfun$ofRows$2(Dataset.scala:103)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:968)
	at org.apache.spark.sql.Dataset$.ofRows(Dataset.scala:101)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$1(SparkSession.scala:803)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:968)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:798)
	at org.apache.spark.sql.SQLContext.sql(SQLContext.scala:695)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.$anonfun$executeSql$1(SQLDriverLocal.scala:91)
	at scala.collection.immutable.List.map(List.scala:293)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.executeSql(SQLDriverLocal.scala:37)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.repl(SQLDriverLocal.scala:145)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$11(DriverLocal.scala:605)
	at com.databricks.logging.Log4jUsageLoggingShim$.$anonfun$withAttributionContext$1(Log4jUsageLoggingShim.scala:28)
	at scala.util.DynamicVariable.withVal

In [0]:
dbutils.fs.ls("/FileStore/tables/source")

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
<command-4388377366863480> in <module>
----> 1 dbutils.fs.ls("/FileStore/tables/source")

/databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling(*args, **kwargs)
    387                     exc.__context__ = None
    388                     exc.__cause__ = None
--> 389                     raise exc
    390 
    391             return f_with_exception_handling

ExecutionError: An error occurred while calling z:com.databricks.backend.daemon.dbutils.FSUtils.ls.
: java.io.FileNotFoundException: /FileStore/tables/source
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:120)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV1.listStatus(DatabricksFileSystemV1.scala:179)
	at com.databri

In [0]:
dbutils.fs.ls("/FileStore/tables/archieve")

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
<command-4388377366863481> in <module>
----> 1 dbutils.fs.ls("/FileStore/tables/archieve")

/databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling(*args, **kwargs)
    387                     exc.__context__ = None
    388                     exc.__cause__ = None
--> 389                     raise exc
    390 
    391             return f_with_exception_handling

ExecutionError: An error occurred while calling z:com.databricks.backend.daemon.dbutils.FSUtils.ls.
: java.io.FileNotFoundException: /FileStore/tables/archieve
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:120)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV1.listStatus(DatabricksFileSystemV1.scala:179)
	at com.dat

In [0]:
dbutils.fs.mv("/FileStore/tables/source", "/FileStore/tables/archieve" ,True)

Out[69]: True